New Scractchbook with just usd from 'Price' column

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import time

# Import visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Import statistical module
from scipy import stats

# Import custom modules
import acquire
import prepare
import util

pd.set_option('display.max_columns', None)

ModuleNotFoundError: No module named 'acquire'

## Wrangle

In [ ]:
# Acquire and prepare data returned as dataframe
df = acquire.get_data(query_url = False)
print(df.shape)
df = prepare.prepare_dataframe(df)
train, validate, test = prepare.split_data(df)
train.shape, validate.shape, test.shape

## Q1: Is there a significant difference in usd between non-foil only cards and non-foil cards with foil versions?

## Key takeaway:
- Taking into account all sets the difference in favor of non-foil only cards is about $10.91

## Discusion:
- The alternate hypothesis that there is a statistically significant difference between the prices of non-foil only cards and cards with both foil & non-foil versions is True.

In [ ]:
# Creates column for cards that have both foil & non-foil versions
train['foil_and_nonfoil'] = train['nonfoil'] == train['foil']

# Shows number of cards and percentage of cards with both versions
train.foil_and_nonfoil.value_counts(), train.foil_and_nonfoil.value_counts(normalize = True)

In [ ]:
# Creates column for cards that have only non-foil versions
train['nonfoil_only'] = (train['nonfoil'] == True) & (train['foil'] == False)

# Shows number of cards and percentage of cards with with only non-foil versions
train.nonfoil_only.value_counts(), train.nonfoil_only.value_counts(normalize = True)

In [ ]:
train.loc[train['nonfoil_only'] == True, 'nonfoil_only_usd'] = train['usd']

In [ ]:
train.foil_and_nonfoil

In [ ]:
avg_nonfoil_only = train.nonfoil_only_usd.mean()
avg_nonfoil_only

In [ ]:
# Fills in NaN values of non-foil only cards with '0'
train = train.fillna(0)

In [ ]:
train.loc[train['foil_and_nonfoil'] == True, 'foil_and_nonfoil_usd'] = train['usd']

In [ ]:
avg_foil_and_nonfoil = train.foil_and_nonfoil_usd.mean()
avg_foil_and_nonfoil

In [ ]:
# Fills in NaN values for foil & nonfoil version with '0'
train = train.fillna(0)

In [ ]:
# Difference in USD of both foil & non-foil from only non-foil
foil_diff = avg_nonfoil_only - avg_foil_and_nonfoil
foil_diff

In [ ]:
train['nonfoil_only_and_both_diff'] = train.nonfoil_only_usd - train.foil_and_nonfoil_usd
train['nonfoil_only_and_both_diff']

In [ ]:
foil_only_vs_both = pd.concat([train.nonfoil_only_usd, train.foil_and_nonfoil_usd, train.nonfoil_only_and_both_diff], axis=1)

In [ ]:
foil_only_vs_both.head()

In [ ]:
def only_foil_vs_both_versions_viz(df):
    plt.figure(figsize = (16,9))
    sns.swarmplot(data = train, x = 'usd')
    plt.title('Difference in Average Price of Only Foils and Both Foils & Non-Foils')
    plt.xlabel('USD')
    plt.ylabel('Only Foil or Both Versions')
    plt.show()

only_foil_vs_both_versions_viz(foil_only_vs_both)

## Hypothesis Testing
Significance Level: α = 0.05
- $H_0$ = There is not a signficant difference between non-foil only cards and cards with both foil & non-foil
- $H_A$ = There is a signficant difference between non-foil only cards and cards with both foil & non-foil

In [ ]:
# Set α to 0.05
α = 0.05
# Checks variance
print(nonfoil_only_usd.var())
print(foil_and_nonfoil_usd.var())

In [ ]:
t, p = stats.ttest_ind(train.foil_and_nonfoil_usd, train.nonfoil_only_usd, equal_var=False)
t, p / 2

In [ ]:
p / 2 < α

## Q2: What’s the difference in card prices of prior printings vs its latest reprint?
- Difference in price for reprinted vs not reprinted

In [ ]:
reprints = train.groupby("reprint")['usd'].mean().dropna()

In [ ]:
reprints
# Figure out how to get the difference progammatically

## Key Takeaway:
- Reprinted cards cost about 77¢ more on average than original prints


## Visualization

## Discusion
- The alternate hypothesis that there is a statistically significant difference between the prices of reprints and original printings is true.

## Hypothesis Testing
Significance Level: α = 0.05
- $H_0$ = There is not a signficant difference between foil & non-foil cards
- $H_A$ = There is a significant difference between foil & non-foil cards

In [ ]:
t, p = stats.ttest_ind(avg_foil, avg_nonfoil, equal_var=False)
t, p / 2

Exploratory Mode

In [ ]:
train.columns

In [ ]:
train['name'].value_counts()

In [ ]:
train['lang'].value_counts()

In [ ]:
train['released_at'].value_counts()

In [ ]:
train['layout'].value_counts()

In [ ]:
train['highres_image'].value_counts()

In [ ]:
train['image_status'].value_counts()

In [ ]:
train['mana_cost'].value_counts()

In [ ]:
train['cmc'].value_counts()

In [ ]:
train['type_line'].value_counts()

In [ ]:
train['oracle_text'].value_counts()

In [ ]:
train['power'].value_counts()

In [ ]:
train['toughness'].value_counts()

In [ ]:
train['colors'].value_counts()
# Separate colors into their own columns if they have that color?

In [ ]:
train['color_identity'].value_counts()
# Should I go based off of color identity or actual casting colors?

In [ ]:
train['keywords'].value_counts()

In [ ]:
train['legalities'].value_counts()

In [ ]:
train['games'].value_counts()

In [ ]:
train['reserved'].value_counts()

In [ ]:
train['foil'].value_counts()

In [ ]:
train['nonfoil'].value_counts()

In [ ]:
train['finishes'].value_counts()

In [ ]:
train['oversized'].value_counts()

In [ ]:
train['promo'].value_counts()